In [ ]:
#############改良箇所#########################
#学習データに対してランダムな回転・ズーム・反転などを適用し、過学習防止のデータ拡張をしました。
#エポックが進むにつれて学習率を動的に減少させ安定させました。
#モデルが画像のどの部分を重視しているのかを表示しました。
###################################################################

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import cv2

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


model = models.Sequential([
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

def lr_schedule(epoch, lr):
    return lr * 0.9 if epoch > 0 else lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels),
                    callbacks=[lr_callback])

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")

def grad_cam(image, model, class_index):
    img_tensor = np.expand_dims(image, axis=0)

    grad_model = tf.keras.models.Model([model.input],
                                       [model.get_layer(index=-3).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    heatmap = cv2.resize(heatmap.numpy(), (32, 32))

    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(cv2.cvtColor(np.uint8(image * 255), cv2.COLOR_RGB2BGR), 0.6, heatmap, 0.4, 0)

    return superimposed_img

sample_image = test_images[0]
predicted_label = np.argmax(model.predict(np.expand_dims(sample_image, axis=0)))
grad_cam_image = grad_cam(sample_image, model, predicted_label)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(sample_image)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(grad_cam_image)
plt.axis('off')

plt.show()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 97s 61ms/step - accuracy: 0.3380 - loss: 1.8016 - val_accuracy: 0.5277 - val_loss: 1.3052 - learning_rate: 0.0010
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 92s 59ms/step - accuracy: 0.4985 - loss: 1.3901 - val_accuracy: 0.5746 - val_loss: 1.1854 - learning_rate: 9.0000e-04
Epoch 3/10
 775/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 60ms/step - accuracy: 0.5455 - loss: 1.2695